In [253]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
import numpy as np
import io
from os.path import isfile, join
from google.cloud.vision import types
from google.cloud import vision
import pandas as pd
import ast
import sys
import json
sys.path.append('../')
from modules.Helpers.dataframe_to_json import DataframeToJSON
from modules.Helpers.JSON_To_DF import JSONToDataframe
from modules.Transformers.CenterPosition import CenterPositionTransform
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from modules.Transformers.TextType import isdateTransform, isfloatTransform,istextTransform

In [104]:
SYMBOL = {
    "$": 0,
}
sfea_words = SYMBOL

def clean_word(word):
    """
    Clean up a word
    :param word: input wordText
    :return: cleaned up word string
    """
    cword = word.strip(' /)*%-_:;\'\"')
    cword = cword.replace(',', '')
    cword = cword.replace('.', '')
    cword = cword.replace('(', '')
    cword = cword.replace('$', '')
    return cword
def gen_raw_feature(words):
    """
    Input a word list from one page, output preliminary features for this page.
    :param words: A list of strings which are words from this page.
    :return: A list of preliminary features contains word frequencies and other features like word count.
                Need some cleanup afterwards.
    """
    word_count = 0
    number_count = 0
    fea_sym = [0]*(max(sfea_words.values())+1)
    for w in words:
        # Count '$'
        dn = w.count('$')
        fea_sym[sfea_words['$']] += dn
        if dn > 1:
            wl = w.split('$')
            w = wl[0]
            words += wl[1:]
        cw = clean_word(w)
        if not cw:
            continue
        # Increase word count
        word_count += 1
        # Check if numbers and increase number count
        if cw.isnumeric():
            number_count += 1
            continue
    # Construct other features
    if not word_count:
        fea_oth = [0, 0, 0]
    else:
        fea_oth = [word_count, number_count]
    return fea_sym + fea_oth

## Image to JSON

In [31]:
def detect_text(path):
    df = pd.DataFrame()
    page = path.split('.')[1].replace('pdf','')
    
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations

    row_dict = {'WordText':None, 'PageNumber': None, 'X_Min': None, 'X_Max': None, 'Y_Min': None, 'Y_Max': None}
    for text in texts:

        row_dict['WordText'] = text.description

        vertices = ([[vertex.x, vertex.y]
                    for vertex in text.bounding_poly.vertices])
        
        xlist = []
        ylist = []
        for vertex in vertices:
            for v in vertex:
                xlist.append(vertex[0])
                ylist.append(vertex[1])
        row_dict['X_Min'] = min(xlist)
        row_dict['X_Max'] = max(xlist)
        row_dict['Y_Min'] = min(ylist)
        row_dict['Y_Max'] = max(ylist)
        
        row_dict['PageNumber'] = page

        df = df.append(row_dict, ignore_index=True)
    df = df[pd.notnull(df['WordText'])]
    return ast.literal_eval(DataframeToJSON(df))

In [37]:
mypath = 'C:/Users/juang/Documents/Stocks/PDFs/'
for i in range(0, len(tickers)):
    json_dict = {'Ticker': None, 'Pages': []}
    json_dict['Ticker'] = tickers[i]
    onlyfiles = [f for f in os.listdir(mypath+tickers[i]) if ".pdf" in f]
    print(onlyfiles)
#     onlyfiles = [s for s in onlyfiles if "all" in s]
    print(tickers[i])
    for f in onlyfiles:
        if '.pdf' in f and 'all' not in onlyfiles:
            json_row = detect_text(mypath+tickers[i]+'/'+f)
            json_dict['Pages'].append(json_row)
    
    with open(mypath+"/"+tickers[i]+"/"+'all'+'.json', 'w') as fp:
        json.dump(json_dict, fp)

['F.pdf0.jpg', 'F.pdf1.jpg', 'F.pdf10.jpg', 'F.pdf11.jpg', 'F.pdf12.jpg', 'F.pdf13.jpg', 'F.pdf14.jpg', 'F.pdf15.jpg', 'F.pdf16.jpg', 'F.pdf17.jpg', 'F.pdf18.jpg', 'F.pdf19.jpg', 'F.pdf2.jpg', 'F.pdf20.jpg', 'F.pdf21.jpg', 'F.pdf22.jpg', 'F.pdf23.jpg', 'F.pdf24.jpg', 'F.pdf25.jpg', 'F.pdf26.jpg', 'F.pdf27.jpg', 'F.pdf28.jpg', 'F.pdf29.jpg', 'F.pdf3.jpg', 'F.pdf30.jpg', 'F.pdf31.jpg', 'F.pdf32.jpg', 'F.pdf33.jpg', 'F.pdf34.jpg', 'F.pdf35.jpg', 'F.pdf36.jpg', 'F.pdf37.jpg', 'F.pdf38.jpg', 'F.pdf39.jpg', 'F.pdf4.jpg', 'F.pdf40.jpg', 'F.pdf41.jpg', 'F.pdf42.jpg', 'F.pdf43.jpg', 'F.pdf44.jpg', 'F.pdf45.jpg', 'F.pdf46.jpg', 'F.pdf47.jpg', 'F.pdf48.jpg', 'F.pdf49.jpg', 'F.pdf5.jpg', 'F.pdf50.jpg', 'F.pdf51.jpg', 'F.pdf52.jpg', 'F.pdf53.jpg', 'F.pdf54.jpg', 'F.pdf55.jpg', 'F.pdf56.jpg', 'F.pdf57.jpg', 'F.pdf58.jpg', 'F.pdf59.jpg', 'F.pdf6.jpg', 'F.pdf60.jpg', 'F.pdf61.jpg', 'F.pdf62.jpg', 'F.pdf63.jpg', 'F.pdf64.jpg', 'F.pdf65.jpg', 'F.pdf66.jpg', 'F.pdf67.jpg', 'F.pdf68.jpg', 'F.pdf69.jpg', '

In [5]:
test = detect_text('wakeupcat.pdf0.jpg')
test = test.reset_index()
intakePipeSteps = []
intakePipeSteps.append(('CenterTransform',CenterPositionTransform()))
intakePipeSteps.append(('isfloat',isfloatTransform('PageText')))
intakePipeSteps.append(('isdate',isdateTransform('PageText')))
intakePipeSteps.append(('istext',istextTransform('PageText')))
combined = Pipeline(intakePipeSteps)
combined
# textcenter = CenterPositionTransform(test)
# textcenter

[{'PageNumber': '0',
  'WordText': 'Wake',
  'X_Max': 211.0,
  'X_Min': 30.0,
  'Y_Max': 458.0,
  'Y_Min': 398.0},
 {'PageNumber': '0',
  'WordText': 'up',
  'X_Max': 308.0,
  'X_Min': 227.0,
  'Y_Max': 464.0,
  'Y_Min': 405.0},
 {'PageNumber': '0',
  'WordText': 'human.',
  'X_Max': 568.0,
  'X_Min': 325.0,
  'Y_Max': 458.0,
  'Y_Min': 398.0}]

In [6]:
print(combined.named_steps['istext'].transform(test))

4 10 istext in
   index PageNumber          PageText  X_Max  X_Min  Y_Max  Y_Min  \
0      0          0  Wake up human.\n  567.0   30.0  464.0  398.0   
1      1          0              Wake  210.0   30.0  458.0  398.0   
2      2          0                up  307.0  227.0  464.0  405.0   
3      3          0            human.  567.0  325.0  458.0  398.0   

   PageText_isdate  PageText_isfloat  PageText_istext  
0                0             False                1  
1                0             False                1  
2                0             False                1  
3                0             False                1  


## PDF to Images

In [21]:
import sys
import PyPDF2
import PythonMagick
from os.path import isfile, join

mypath = 'C:/Users/juang/Documents/Stocks/PDFs/'
onlyfiles = [f for f in os.listdir(mypath) if isfile(join(mypath, f))]
for f in onlyfiles:
    directory = 'C:/Users/juang/Documents/Stocks/PDFs/'+str(f).replace('.pdf','')
    if not os.path.exists(directory):
        os.makedirs(directory)
        pdffilename = f 
        pdf_im = PyPDF2.PdfFileReader(open('C:/Users/juang/Documents/Stocks/PDFs/'+pdffilename, "rb"))
        npage = pdf_im.getNumPages()
        print('Converting %d pages.' % npage)
        for p in range(npage):
            im = PythonMagick.Image()
            im.density('300')
            im.read('C:/Users/juang/Documents/Stocks/PDFs/'+pdffilename + '[' + str(p) +']')
            im.write(directory+'/'+pdffilename + str(p)+ '.jpg')
    elif not len(os.listdir(directory)) >= 1:
        pdffilename = f 
        pdf_im = PyPDF2.PdfFileReader(open('C:/Users/juang/Documents/Stocks/PDFs/'+pdffilename, "rb"))
        npage = pdf_im.getNumPages()
        print('Converting %d pages.' % npage)
        for p in range(npage):
            im = PythonMagick.Image()
            im.density('300')
            im.read('C:/Users/juang/Documents/Stocks/PDFs/'+pdffilename + '[' + str(p) +']')
            im.write(directory+'/'+pdffilename + str(p)+ '.jpg')

Converting 30 pages.


## HTML to PDF

In [28]:
import os.path
import pdfkit

tickers = []
config = pdfkit.configuration(wkhtmltopdf='C:\Program Files\wkhtmltopdf\\bin\\wkhtmltopdf.exe')
files = [f for f in os.listdir('.') if os.path.isfile(f)]
for f in files:
    tickers.append(f.split('-')[0])
    if '.html' in f and not os.path.isfile('C:\\Users\\juang\\Documents\\Stocks\\PDFs\\'+f.split('-')[0]+'.pdf'): 
        print(f.split('-')[0]+'.pdf')
        pdfkit.from_file(f, 'C:\\Users\\juang\\Documents\\Stocks\\PDFs\\'+f.split('-')[0]+'.pdf', configuration=config)

## Read JSON Pages/Financials Classifier

In [38]:
pd.set_option('display.max_rows', 500)

In [392]:
with open('C:\\Users\\juang\\Documents\\Stocks\\PDFs\\F\\all.json', 'r') as f:
    datastore = json.load(f)
df_f = pd.DataFrame()
for i in range(0, len(datastore['Pages'])):
    df_f = df_f.append(pd.DataFrame(datastore['Pages'][i]))
df_allf = df_f[df_f.index == 0].set_index('PageNumber')
df_allf.index = df_allf.index.astype(int)
df_allf = df_allf.sort_index()
df_allf.loc[:,'Label'] = 0
df_allf['Label'][3:7] = 1
df_allf['Dollar_Count'] = ''
df_allf['Word_Count'] = ''
df_allf['Number_Count'] = ''
for i in range(0, len(df_allf)):
    df_allf['Dollar_Count'][i] = gen_raw_feature(df_allf.WordText[i])[0]
    df_allf['Word_Count'][i] = gen_raw_feature(df_allf.WordText[i])[1]
    df_allf['Number_Count'][i] = gen_raw_feature(df_allf.WordText[i])[2]
df_allf

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: Setting

,WordText,X_Max,X_Min,Y_Max,Y_Min,Label,Dollar_Count,Word_Count,Number_Count
PageNumber,,,,,,,,,
0,10-Q 1 f0930201710-q.htm 10-Q\nUNITED STATES\n...,2332.0,138.0,2552.0,123.0,0,1,2012,132
1,FORD MOTOR COMPANY\nQUARTERLY REPORT ON FORM 1...,2269.0,143.0,2073.0,238.0,0,0,985,38
2,PART I. FINANCIAL INFORMATION\nITEM 1. Financi...,2320.0,140.0,2892.0,164.0,0,11,1630,256
3,Item 1. Financial Statements (Continued)\nFORD...,2324.0,143.0,3267.0,123.0,1,9,1949,330
4,Item 1. Financial Statements (Continued)\nFORD...,2327.0,143.0,2066.0,123.0,1,4,1229,183
5,Item 1. Financial Statements (Continued)\nFORD...,2328.0,138.0,1753.0,123.0,1,25,1068,235
6,Item 1. Financial Statements (Continued)\nFORD...,2258.0,143.0,1264.0,123.0,1,0,712,55
7,Item 1. Financial Statements (Continued)\nFORD...,2342.0,138.0,1875.0,123.0,0,0,3190,58
8,Item 1. Financial Statements (Continued)\nFORD...,2320.0,138.0,1770.0,123.0,0,4,1410,200


In [393]:
with open('C:\\Users\\juang\\Documents\\Stocks\\PDFs\\FB\\all.json', 'r') as f:
    datastore = json.load(f)
df_fb = pd.DataFrame()
for i in range(0, len(datastore['Pages'])):
    df_fb = df_fb.append(pd.DataFrame(datastore['Pages'][i]))
df_allfb = df_fb[df_fb.index == 0].set_index('PageNumber')
df_allfb.index = df_allfb.index.astype(int)
df_allfb = df_allfb.sort_index()
df_allfb.loc[:,'Label'] = 0
df_allfb['Label'][5:10] = 1
df_allfb['Dollar_Count'] = ''
df_allfb['Word_Count'] = ''
df_allfb['Number_Count'] = ''
for i in range(0, len(df_allfb)):
    df_allfb['Dollar_Count'][i] = gen_raw_feature(df_allfb.WordText[i])[0]
    df_allfb['Word_Count'][i] = gen_raw_feature(df_allfb.WordText[i])[1]
    df_allfb['Number_Count'][i] = gen_raw_feature(df_allfb.WordText[i])[2]
df_allfb

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: Setting

,WordText,X_Max,X_Min,Y_Max,Y_Min,Label,Dollar_Count,Word_Count,Number_Count
PageNumber,,,,,,,,,
0,10-Q 1 fb-09302017x10q.htm 10-0\nUNITED STATES...,2332.0,138.0,2328.0,123.0,0,0,2260,153
1,"FACEBOOK, INC.\nTABLEOF CONTENTS\nLimitations ...",2337.0,142.0,1554.0,164.0,0,0,247,10
2,NOTEABOUTFORWARD-LOOKINGSTATEMENTS\nThis Quart...,2341.0,138.0,1027.0,193.0,0,0,2305,13
3,LIMITATIONS OF KEY METRICS AND OTHER DATA\nThe...,2339.0,135.0,1967.0,193.0,0,0,4696,28
4,PARTI-FINANCIAL INFORMATION\nItem1. Financial ...,2324.0,140.0,2255.0,162.0,0,2,1568,262
5,"FACEBOOK, INC.\nCONDENSED CONSOLIDATED STATEME...",2320.0,143.0,1936.0,164.0,1,8,1266,272
6,"FACEBOOK, INC.\nCONDENSED CONSOLIDATED STATEME...",2337.0,143.0,829.0,164.0,1,5,464,69
7,"FACEBOOK, INC.\nCONDENSED CONSOLIDATED STATEME...",2329.0,143.0,2280.0,164.0,1,2,1535,177
8,"FACEBOOK, INC.\nCONDENSED CONSOLIDATED STATEME...",2321.0,143.0,975.0,164.0,1,2,553,28


In [394]:
with open('C:\\Users\\juang\\Documents\\Stocks\\PDFs\\RAD\\all.json', 'r') as f:
    datastore = json.load(f)
df_rad = pd.DataFrame()
for i in range(0, len(datastore['Pages'])):
    df_rad = df_rad.append(pd.DataFrame(datastore['Pages'][i]))
df_allrad = df_rad[df_rad.index == 0].set_index('PageNumber')
df_allrad.index = df_allrad.index.astype(int)
df_allrad = df_allrad.sort_index()
df_allrad.loc[:,'Label'] = 0
df_allrad['Label'][3:7] = 1
df_allrad['Dollar_Count'] = ''
df_allrad['Word_Count'] = ''
df_allrad['Number_Count'] = ''
for i in range(0, len(df_allrad)):
    df_allrad['Dollar_Count'][i] = gen_raw_feature(df_allrad.WordText[i])[0]
    df_allrad['Word_Count'][i] = gen_raw_feature(df_allrad.WordText[i])[1]
    df_allrad['Number_Count'][i] = gen_raw_feature(df_allrad.WordText[i])[2]
df_allrad

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: Setting

,WordText,X_Max,X_Min,Y_Max,Y_Min,Label,Dollar_Count,Word_Count,Number_Count
PageNumber,,,,,,,,,
0,10-Q 1 al7-20664 110q.htm 10-Q\nTable of Conte...,2328.0,138.0,3359.0,125.0,0,0,2131,125
1,ns\ntions for the Thi\nek Pe\n2017 and Augus\n...,2337.0,138.0,3370.0,162.0,0,0,3285,44
2,* our ability to manage expenses and working c...,2334.0,138.0,3373.0,166.0,0,2,3374,167
3,"1,453,291\nDeferred tax assets\nOtherassets\n1...",2327.0,137.0,3373.0,113.0,1,0,2114,367
4,See accompanying notes to condensed consolidat...,2308.0,137.0,3378.0,167.0,1,5,1669,241
5,"Lease termination and impairment charges\n13,0...",2334.0,137.0,3362.0,123.0,1,5,4131,206
6,"the twenty-six weeks ended September 2, 2017, ...",2337.0,138.0,3347.0,123.0,1,1,6501,252
7,Total\nFinal purchase price allocatior\nCash a...,2339.0,138.0,3313.0,123.0,0,0,4252,250
8,The parties to the Amended and Restated Asset ...,2337.0,138.0,3330.0,121.0,0,2,4182,207


In [454]:
with open('C:\\Users\\juang\\Documents\\Stocks\\PDFs\\UPS\\all.json', 'r') as f:
    datastore = json.load(f)
df_ups = pd.DataFrame()
for i in range(0, len(datastore['Pages'])):
    df_ups = df_ups.append(pd.DataFrame(datastore['Pages'][i]))
df_allups = df_ups[df_ups.index == 0].set_index('PageNumber')
df_allups.index = df_allups.index.astype(int)
df_allups = df_allups.sort_index()
df_allups.loc[:,'Label'] = 0
df_allups['Label'][4:7] = 1
df_allups['Dollar_Count'] = ''
df_allups['Word_Count'] = ''
df_allups['Number_Count'] = ''
for i in range(0, len(df_allups)):
    df_allups['Dollar_Count'][i] = gen_raw_feature(df_allups.WordText[i])[0]
    df_allups['Word_Count'][i] = gen_raw_feature(df_allups.WordText[i])[1]
    df_allups['Number_Count'][i] = gen_raw_feature(df_allups.WordText[i])[2]
df_allups

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: Setting

,WordText,X_Max,X_Min,Y_Max,Y_Min,Label,Dollar_Count,Word_Count,Number_Count
PageNumber,,,,,,,,,
0,10-Q 1 ups-09302017xl0q.htm FORM 10-Q\nUnited ...,2323.0,138.0,2421.0,123.0,0,0,2145,139
1,"UNITED PARCELSERVICE, INC.\nQUARTERLY REPORT O...",2337.0,142.0,2765.0,196.0,0,0,305,38
2,PARTL FINANCIAL INFORMATION\nCa\nutionary Stat...,2330.0,138.0,1421.0,196.0,0,0,3611,32
3,tem1. Financial Statements\nUNITED PARCELSERVI...,2324.0,140.0,2654.0,195.0,0,1,1591,264
4,"UNITED PARCELSERVICE, INC. AND SUBSIDIARIES\nS...",2320.0,142.0,2318.0,196.0,1,4,1289,246
5,"UNITED PARCELSERVICE, INC. AND SUBSIDIARIES\nS...",2330.0,143.0,2717.0,196.0,1,0,1663,180
6,"UNITED PARCELSERVICE, INC. AND SUBSIDIARIES\nN...",2332.0,135.0,1936.0,196.0,1,0,4045,62
7,"UNITED PARCELSERVICE, INC. AND SUBSIDIARIES\nN...",2336.0,137.0,2041.0,155.0,0,1,4609,85
8,"UNITED PARCELSERVICE, INC. AND SUBSIDIARIES\nN...",2336.0,138.0,1958.0,155.0,0,1,5013,87


In [455]:
df_all = df_allf.append(df_allfb)
df_all = df_all.append(df_allrad, ignore_index=True)
df_all = df_all.append(df_allups, ignore_index=True)
df_all = df_all.append(df_allgme, ignore_index=True)
df_all

,Dollar_Count,Label,Number_Count,WordText,Word_Count,X_Max,X_Min,Y_Max,Y_Min
0,1,0,132,10-Q 1 f0930201710-q.htm 10-Q\nUNITED STATES\n...,2012,2332.0,138.0,2552.0,123.0
1,0,0,38,FORD MOTOR COMPANY\nQUARTERLY REPORT ON FORM 1...,985,2269.0,143.0,2073.0,238.0
2,11,0,256,PART I. FINANCIAL INFORMATION\nITEM 1. Financi...,1630,2320.0,140.0,2892.0,164.0
3,9,1,330,Item 1. Financial Statements (Continued)\nFORD...,1949,2324.0,143.0,3267.0,123.0
4,4,1,183,Item 1. Financial Statements (Continued)\nFORD...,1229,2327.0,143.0,2066.0,123.0
5,25,1,235,Item 1. Financial Statements (Continued)\nFORD...,1068,2328.0,138.0,1753.0,123.0
6,0,1,55,Item 1. Financial Statements (Continued)\nFORD...,712,2258.0,143.0,1264.0,123.0
7,0,0,58,Item 1. Financial Statements (Continued)\nFORD...,3190,2342.0,138.0,1875.0,123.0
8,4,0,200,Item 1. Financial Statements (Continued)\nFORD...,1410,2320.0,138.0,1770.0,123.0
9,6,0,256,Item 1. Financial Statements (Continued)\nFORD...,1348,2320.0,140.0,2401.0,123.0


In [466]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB(fit_prior=False, alpha=0.01)),])
text_clf.fit(df_all.WordText, df_all.Label) 

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=0.01, class_prior=None, fit_prior=False))])

In [467]:
array = [df_all.Number_Count/df_all.Word_Count,df_all.Dollar_Count]
np.transpose(array)
number_clf = Pipeline([('clf', GaussianNB())])
number_clf.fit(np.transpose(array), df_all.Label)
gaussian_probas = number_clf.predict_proba(np.transpose(array))
multinomial_probas = text_clf.predict_proba(df_all.WordText)
all_probas = np.hstack((multinomial_probas, gaussian_probas))
final_clf = Pipeline([('clf', GaussianNB())])
final_clf.fit(all_probas, df_all.Label)

Pipeline(steps=[('clf', GaussianNB(priors=None))])

In [468]:
array1 = [df_allgme.Number_Count/df_allgme.Word_Count, df_allgme.Dollar_Count]
g = number_clf.predict_proba(np.transpose(array1))
m = text_clf.predict_proba(df_allgme.WordText)
all_probas = np.hstack((m, g))
all_probas
final_clf.predict(all_probas)

array([0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [470]:
array1 = [df_all.Number_Count/df_all.Word_Count, df_all.Dollar_Count]
g = number_clf.predict_proba(np.transpose(array1))
m = text_clf.predict_proba(df_all.WordText)
all_probas = np.hstack((m, g))
predicted = final_clf.predict(all_probas)
labels = ['Other', 'Financial']
print(metrics.classification_report(df_all.Label, predicted, target_names=labels))

             precision    recall  f1-score   support

      Other       1.00      0.99      0.99       243
  Financial       0.88      1.00      0.93        21

avg / total       0.99      0.99      0.99       264



In [453]:
with open('C:\\Users\\juang\\Documents\\Stocks\\PDFs\\GME\\all.json', 'r') as f:
    datastore = json.load(f)
df_gme = pd.DataFrame()
for i in range(0, len(datastore['Pages'])):
    df_gme = df_gme.append(pd.DataFrame(datastore['Pages'][i]))
df_gme
df_allgme = df_gme[df_gme.index == 0].set_index('PageNumber')
df_allgme.index = df_allgme.index.astype(int)
df_allgme = df_allgme.sort_index()
df_allgme['Dollar_Count'] = ''
df_allgme['Word_Count'] = ''
df_allgme['Number_Count'] = ''
df_allgme.loc[:,'Label'] = 0
df_allgme['Label'][2:7] = 1
for i in range(0, len(df_allgme)):
    df_allgme['Dollar_Count'][i] = gen_raw_feature(df_allgme.WordText[i])[0]
    df_allgme['Word_Count'][i] = gen_raw_feature(df_allgme.WordText[i])[1]
    df_allgme['Number_Count'][i] = gen_raw_feature(df_allgme.WordText[i])[2]
df_allgme

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the

,WordText,X_Max,X_Min,Y_Max,Y_Min,Dollar_Count,Word_Count,Number_Count,Label
PageNumber,,,,,,,,,
0,10-Q 1 al0q-fy17q3.htm 10-Q\nUNITED STATES SEC...,2339.0,138.0,2407.0,123.0,1,2047,126,0
1,"TABLEOFCONTENTS\nPage No.\nItem 1.\nmen\n28,20...",2340.0,143.0,1736.0,161.0,0,176,34,0
2,PARTI FINANCIAL INFORMATION\nITEM 1 ANANCIAL S...,2322.0,140.0,2613.0,198.0,7,1518,318,1
3,GAMESTOP CORP\nUNAUDITED CONDENSED CONSOLDATED...,2329.0,142.0,2366.0,196.0,13,859,259,1
4,GAMESTOP CORP\nUNAUDITED CONDENSED CONSOLIDATE...,2319.0,143.0,2186.0,196.0,6,391,67,1
5,GAMESTOP CORP\nUNAUDITED CONDENSED CONSOLIDATE...,2329.0,145.0,2339.0,196.0,17,1053,187,1
6,GAMESTOP CORP\nUNAUDITED CONDENSED CONSOLIDATE...,2329.0,143.0,2618.0,196.0,2,1686,170,1
7,GAMESTOP CORP\nCONDENSED NOTES TO UNAUDITED CO...,2342.0,137.0,2275.0,196.0,4,4546,128,0
8,GAMESTOP CORP\nCONDENSED NOTES TO UNAUDITED CO...,2340.0,138.0,2203.0,157.0,0,5657,114,0
